<a href="https://colab.research.google.com/github/GutembergPraia/webscraping_tripadvisor/blob/main/webscraping_tripadvisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo - Atraves de uma URL de pequisa no tripadvisor, percorre todos os resultados, capturando os comentarios, datada de postagem

Instalação da biblioteacas necessarias:  

In [1]:
pip install beautifulsoup4

In [2]:
pip install requests

In [3]:
pip install pandas

In [5]:
def mes_to_int(mes: str):
    mes_do_ano = {
        'janeiro':'1',
        'fevereiro':'2',
        'março':'3',
        'abril':'4',
        'maio':'5',
        'junho':'6',
        'julho':'7',
        'agosto':'8',
        'setembro':'9',
        'outubro':'10',
        'novembro':'11',
        'dezembro':'12'
    }

    return mes_do_ano[mes]

In [8]:
#importes das bibliotecas
import pandas as pd
import requests
import re

from datetime import datetime
from bs4 import BeautifulSoup
from google.colab import files


#URL da pesquisa
url = 'https://www.tripadvisor.com.br/Attractions-g303235-Activities-c49-t28,30,32,33,161-zft11292-a_sort.TRAVELER__5F__RANKED-Manaus_Amazon_River_State_of_Amazon.html'

#dicionario com os dados que seram coletados:
dic_comentario = {'local':[], 'autor':[], 'nota':[],'titulo':[], 'comentario':[], 'cidade':[], 'estado':[], 'data':[], 'curtidas':[]}

headers ={"user-agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

site = requests.get(url,headers=headers)

soup = BeautifulSoup(site.content, 'html.parser')

In [9]:
#laço de repetição que percorrer todos os resultados da pequisa, capturandos as urls de cada resultado
for urlLocal in soup.find_all("a",attrs={"class":"BUupS _R w _Z y M0 B0 Gm wSSLS"}):
    #acesso a urls do resultado da pesquisa
    urlCompleta = 'https://www.tripadvisor.com.br' + urlLocal.get('href')

    site = requests.get(urlCompleta,headers=headers)
    soupLocal = BeautifulSoup(site.content, 'html.parser')

    #Verificar quantidade total de comentarios
    quantTotal = int(soupLocal.find(attrs={"class":"Ci"}).get_text().split()[3])

    print(quantTotal)

    urlPadrao = []
    urlPadrao.append(urlCompleta.split("Reviews-")[0] + "Reviews-or")
    urlPadrao.append("-" + urlCompleta.split("Reviews-")[1])

    quantAtual = 0

    #laço de repetição que verifica se todos os cometarios foram lidos
    while quantTotal > quantAtual:
        #laço de repetição que percorre todos os reviewCard de uma pagina
        for link in soupLocal.find_all(attrs={"data-automation":"reviewCard"}):
            dic_comentario['local'].append(soupLocal.find(attrs={"class":"biGQs _P fiohW eIegw"}).get_text())
            dic_comentario['autor'].append(link.find("span").get_text())
            dic_comentario['nota'].append(link.find(attrs={"class":"UctUV d H0"}).get("aria-label").split()[0])
            dic_comentario['titulo'].append(link.find(attrs={"class":"biGQs _P fiohW qWPrE ncFvv fOtGX"}).get_text())
            dic_comentario['comentario'].append(link.find(attrs={"class":"fIrGe _T bgMZj"}).get_text())
            data_postagem_str = (link.find(string=re.compile("Feita em"),attrs={"class":"biGQs _P pZUbB ncFvv osNWb"}).get_text()).split()
            data_postagem_str_format = data_postagem_str[2]+'-'+mes_to_int(data_postagem_str[4])+'-'+data_postagem_str[6]
            dic_comentario['data'].append(datetime.strptime(data_postagem_str_format, '%d-%m-%Y'))
            dic_comentario['curtidas'].append(link.find(attrs={"class":"biGQs _P FwFXZ"}).get_text())
            #coleta dado cidade e estado
            cidade_estado = link.find(attrs={"class":"biGQs _P pZUbB osNWb"}).find("span").get_text()
            cidade_estado = cidade_estado.split(", ")
            if(len(cidade_estado)==2):
                dic_comentario['cidade'].append(cidade_estado[0])
                dic_comentario['estado'].append(cidade_estado[1])
            else:
                dic_comentario['cidade'].append("null")
                dic_comentario['estado'].append("null")

        quantAtual += 10
        # se todos os comentarios ainda não foram lidos, carrega a proxima pagina de comentario
        if(quantAtual < quantTotal):

            urlCompleta = urlPadrao[0] + str(quantAtual) + urlPadrao[1]
            #print(urlCompleta)

            site = requests.get(urlCompleta,headers=headers)
            soupLocal = BeautifulSoup(site.content, 'html.parser')

df = pd.DataFrame(data=dic_comentario)

281
270
145
270
113
42
33
30
20
14
18
20
12


In [10]:
#fazer download do arquivo Gerado
#print(dic_comentario)
df = pd.DataFrame(data=dic_comentario)
df.to_csv('output.csv', encoding = 'utf-8-sig')
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>